In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.keras.datasets import mnist
from tensorflow.contrib.eager.python import tfe

D:\Users\Yue\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# enable eager mode
tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [3]:
if not os.path.exists('weights/'):
    os.makedirs('weights/')

# constants
units = 128
batch_size = 100
epochs = 2
num_classes = 10

In [4]:
# dataset loading
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 28, 28))  # 28 timesteps, 28 inputs / timestep
x_test = x_test.reshape((-1, 28, 28))  # 28 timesteps, 28 inputs / timeste

# one hot encode the labels. convert back to numpy as we cannot use a combination of numpy
# and tensors as input to keras
y_train_ohe = tf.one_hot(y_train, depth=num_classes).numpy()
y_test_ohe = tf.one_hot(y_test, depth=num_classes).numpy()

print('x train', x_train.shape)
print('y train', y_train_ohe.shape)
print('x test', x_test.shape)
print('y test', y_test_ohe.shape)

x train (60000, 28, 28)
y train (60000, 10)
x test (10000, 28, 28)
y test (10000, 10)


# Fast LSTM Cell

Here, we take a middle ground approach to the canonical (6.1) slow approach and the custom (6.2) approach to building RNNs. 

It is not recommended to try building RNNs from scratch unless you know what is going on in the model and many factors like initialization, order of operations, activation function can affect results drastically (it is not as fun as it looks). 

Hence, when you want something pre-built, but with the speed comparable to a custom model, you can use the Cell variant of that RNN layer. 

Here, we will use the LSTMCell from Keras to demonstrate this. Note, that this is a full fledged LSTM Cell from Keras, therefore it has a lot more niceties like dropout, recurrent dropout and initializers, constraints and regularizers etc. 

## Notes

A point to take not of is that since a cell was originally meant to be built by the RNN that wraps it, we have to perform an extra check inside `call` to build the RNN if it has not been built the first time. This will make the first call to the Model slightly slower, but not by much.

In essence, we override the K.rnn() symbolic loop and use Pythonic loop to manage the internal state of the lstm as well as the feeding of data slices to it.

It won't take much to extend this to Multi layer RNN. 

- Here we have 2 initial states for 1 layer. For k layers, you would need a list of 2 * k initial states. This is specific to an LSTM. A GRU needs only 1 state.
- Loop around the 1st Cell and manage its output embedding. Here, we wont maintain just the final `x`, but the entire list of `x` over all timesteps, and use `tf.concat(x_list, axis=1)` to get back a time series of shape (batchsize, timesteps, hiddendim)
- After this first layer, use the outer loop to switch to the next cell. Initially, the states of this cell will be the two zero states corresponding to the i*2-th location in the state list.
- Loop around while maintaining all of its intermediate `x` and do the same as above.
- Finally, after all the layers of the RNN are done, feed only the final x of the final Cell to the classifier.

## Noticeable speed difference

Even with the loop using Python alone, this model is slightly slower than the barebones BasicLSTM model. I have no idea why this is the case. Fundamentally, what I wrote for `BasicLSTM` is actually just a LSTM cell, which we use directly here. When I use the Keras `LSTMCell`, it should be equivalent, without the excess stuff like dropout etc. That alone should not account for such a speedup.

**NOTE:** 
If the GPU is available, you should use `implementation=2` for maximum speed on the GPU. This, however, reduces regularization somewhat so be careful about finding good values for `dropout` and `recurrent_dropout`

In [5]:
class RNN(tf.keras.Model):
    def __init__(self, units, num_classes):
        super(RNN, self).__init__()
        self.units = units
        self.lstm_cell = tf.nn.rnn_cell.LSTMCell(units)  # use the tensorflow cell directly
        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=None, mask=None):
        state = self.lstm_cell.zero_state(batch_size=inputs.shape[0], dtype=tf.float32)
        x = inputs

        # if the LSTMCell is being called for the first time ever
        # Build the cell's weights manually
        if not self.lstm_cell.built:
            self.lstm_cell._dtype = tf.float32  # important : force set the data type via its private variable
            self.lstm_cell.build(inputs.shape[1:])

        for t in range(inputs.shape[1]):
            input = inputs[:, t, :]  # extract the current input at timestep t
            x, state = self.lstm_cell(input, state=state)  # get the output embedding and the states ; note `state` rather than `states`

            # states = feed in the states back to the next timestep

        output = self.classifier(x)  # feed the last `x` as the hidden embedding of the lstm to the classifier

        # softmax op does not exist on the gpu, so always use cpu
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(output)

        return output

In [6]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'

with tf.device(device):
    # build model and optimizer
    model = RNN(units, num_classes)
    model.compile(optimizer=tf.train.AdamOptimizer(0.01), loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # TF Keras tries to use entire dataset to determine shape without this step when using .fit()
    # Fix = Use exactly one sample from the provided input dataset to determine input/output shape/s for the model
    dummy_x = tf.zeros((1, 28, 28))
    model.call(dummy_x)

    # train
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test_ohe), verbose=1)

    # evaluate on test set
    scores = model.evaluate(x_test, y_test_ohe, batch_size, verbose=1)
    print("Final test loss and accuracy :", scores)

    saver = tfe.Saver(model.variables)
    saver.save('weights/06_03_rnn/weights.ckpt')

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 102s 2ms/step - loss: 0.3065 - acc: 0.9026 - val_loss: 0.1253 - val_acc: 0.9624
Epoch 2/2
10000/10000 [==============================] - 4s 436us/step
Final test loss and accuracy : [0.07217171450378373, 0.9790000069141388]
